## ReMKiT1D input generator - testing multilinear interpolation

This example tests the n-dimensional linear interpolation derivation in ReMKiT1D using the json interface

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.IO_support as io 
import RMK_support.simple_containers as sc
from scipy.interpolate import RegularGridInterpolator

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_nd_interp/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Grid setup

In [ ]:
xGrid = np.ones(4) 
gridObj = Grid(xGrid,interpretXGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Add custom derivations

These are the interpolation derivations to be tested

In [ ]:
# grids
grid1 = np.linspace(1,6,10)
grid2 = np.linspace(10,173,15)
grid3 = np.linspace(0.5,3,10)

# 1D interpolation 

data1D = grid1*0.5 + grid1**2/10
rk.addCustomDerivation("interp1D",sc.nDInterpolationDerivation([grid1],data1D))

# 2D interpolation

xg2,yg2 = np.meshgrid(grid1,grid2,indexing='ij',sparse=True)
f = lambda x,y: x**2 + 2*y + 1
data2D = f(xg2,yg2)
rk .addCustomDerivation("interp2D",sc.nDInterpolationDerivation([grid1,grid2],data2D))

# 3D interpolation

xg3,yg3,zg3 = np.meshgrid(grid1,grid2,grid3,indexing='ij',sparse=True)
f = lambda x,y,z: x**2 + 2*y + 1 + z**3
data3D = f(xg3,yg3,zg3)
rk .addCustomDerivation("interp3D",sc.nDInterpolationDerivation([grid1,grid2,grid3],data3D))

In [ ]:
interp1 = RegularGridInterpolator((grid1,),data1D)
interp2 = RegularGridInterpolator((grid1,grid2),data2D)
interp3 = RegularGridInterpolator((grid1,grid2,grid3),data3D)

### Variables

In [ ]:
var1 = np.random.rand(4)*5 + 1
var2 = np.random.rand(4)*163 + 10
var3 = np.random.rand(4)*2.5 + 0.5

rk.addVar("var1",var1,isDerived=True)
rk.addVar("var2",var2,isDerived=True)
rk.addVar("var3",var3,isDerived=True)

rk.addVar("interp1",isDerived=True, derivationRule=sc.derivationRule("interp1D",["var1"]))
rk.addVar("interp2",isDerived=True, derivationRule=sc.derivationRule("interp2D",["var1","var2"]))
rk.addVar("interp3",isDerived=True, derivationRule=sc.derivationRule("interp3D",["var1","var2","var3"]))

In [ ]:
interpVals1 = interp1(var1)
interpVals2 = interp2(np.array(list(zip(var1,var2))))
interpVals3 = interp3(np.array(list(zip(var1,var2,var3))))


In [ ]:

integrator = sc.rkIntegrator(1)
rk.addIntegrator("RK", integrator)


In [ ]:
initialTimestep = 0.0
rk.setIntegratorGlobalData(1, 1, initialTimestep)

In [ ]:
bdeStep = sc.IntegrationStep("RK")

rk.addIntegrationStep("RK1", bdeStep.dict())


#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(1)
rk.setFixedStepOutput(1)


### Create config file

NOTE: Run in build mode, as debug will complain there are no models

In [ ]:
rk.writeConfigFile()


### Data analysis


In [ ]:
numFiles = 1


#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]


In [ ]:
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData


In [ ]:
interpVals1-loadedData["interp1"][-1,:]

In [ ]:
interpVals2-loadedData["interp2"][-1,:]

In [ ]:
interpVals3-loadedData["interp3"][-1,:]